In [1]:
import chainer
from chainer.functions import caffe
from chainer import cuda
import chainer.functions as F
from chainer.functions import caffe
from chainer import cuda, Function, FunctionSet, gradient_check, Variable, optimizers

In [2]:
import json
import unicodecsv as csv
import pickle
import numpy as np
import cv2
from tqdm import tqdm

In [3]:
from gensim import corpora

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
with open('../data/relationships-half.json', 'r') as f:
    all_relationships = json.load(f)
with open('../data/attributes-half.json', 'r') as f:
    all_attributes = json.load(f)
with open('../data/objects.json', 'r') as f:
    all_objects = json.load(f)

In [6]:
with open('../work/relationship_train_image_ids.csv', 'r') as f:
    reader = csv.reader(f)
    relationship_train_image_ids=reader.next()
    relationship_train_image_ids = map(int,relationship_train_image_ids)
    
with open('../work/relationship_val_image_ids.csv', 'r') as f:
    reader = csv.reader(f)
    relationship_val_image_ids=reader.next()
    relationship_val_image_ids = map(int,relationship_val_image_ids)

with open('../work/relationship_test_image_ids.csv', 'r') as f:
    reader = csv.reader(f)
    relationship_test_image_ids=reader.next()
    relationship_test_image_ids = map(int,relationship_test_image_ids)

with open('../work/attribute_train_image_ids.csv', 'r') as f:
    reader = csv.reader(f)
    attribute_train_image_ids=reader.next()
    attribute_train_image_ids = map(int,attribute_train_image_ids)
    
with open('../work/attribute_val_image_ids.csv', 'r') as f:
    reader = csv.reader(f)
    attribute_val_image_ids=reader.next()
    attribute_val_image_ids = map(int,attribute_val_image_ids)

with open('../work/attribute_test_image_ids.csv', 'r') as f:
    reader = csv.reader(f)
    attribute_test_image_ids=reader.next()
    attribute_test_image_ids = map(int,attribute_test_image_ids)

In [7]:
def get_image_relationships(image_id,all_relationships):
    for i in xrange(len(all_relationships)):
        image_id_=all_relationships[i][u'id']
        if image_id==image_id_:
            relationships=all_relationships[i][u'relationships']
    return relationships

def get_image_attributes(image_id,all_attributes):
    for i in xrange(len(all_attributes)):
        image_id_=all_objects[i][u'id']
        if image_id==image_id_:
            attributes=all_attributes[i][u'attributes']
    return attributes

In [8]:
gpu_id=-1
#Load Caffe Model
cnn_model_place="../chainer_caption_generation/data/bvlc_googlenet_caffe_chainer.pkl"
with open(cnn_model_place, 'r') as f:
    func = pickle.load(f)
if gpu_id>= 0:
    func.to_gpu()
print "done"

done


In [9]:
# train_attributes=[]
# for image_id in tqdm(attribute_train_image_ids):
#     image_attributes=get_image_attributes(image_id,all_attributes)
#     for triple in image_attributes:
#         triple_id=triple[u'id']
#         triple_attritbues=triple[u'attributes']
#         for attribute in triple_attritbues:
#             train_attributes.append([attribute])

 15%|█▌        | 6930/44741 [04:23<23:55, 26.34it/s]


KeyboardInterrupt: 

In [ ]:
dictionary = corpora.Dictionary(train_attributes)
print len(dictionary)
fname="../work/attribute_dic_freq.txt"
dictionary.save_as_text(fname, sort_by_word=False)
dictionary.filter_extremes(no_below=5, no_above=1.0)
dictionary.compactify() # remove gaps in id sequence after words that were removed
index2token = dict((v, k) for k, v in dictionary.token2id.iteritems())
ukn_id=len(dictionary.token2id)
index2token[ukn_id]='unknown'
print len(index2token)

# with open('../work/index2attribute.pkl', 'w') as f:
#     pickle.dump(index2token,f)
#relation2index = dict((v, k) for k, v in index2relation.iteritems())

In [10]:
train_relations=[]
for image_id in tqdm(relationship_train_image_ids):
    image_relationships=get_image_relationships(image_id,all_relationships)
    for triple in image_relationships:
        triple_id=triple[u'id']
        triple_predicate=triple[u'predicate']
        train_relations.append([triple_predicate])

  0%|          | 67/42728 [00:02<23:48, 29.86it/s]


KeyboardInterrupt: 

In [ ]:
dictionary = corpora.Dictionary(train_relations)
print len(dictionary)
fname="../work/relation_dic_freq.txt"
dictionary.save_as_text(fname, sort_by_word=False)
dictionary.filter_extremes(no_below=5, no_above=1.0)
dictionary.compactify() # remove gaps in id sequence after words that were removed
index2token = dict((v, k) for k, v in dictionary.token2id.iteritems())
ukn_id=len(dictionary.token2id)
index2token[ukn_id]='unknown'
print len(index2token)

# with open('../work/index2relation.pkl', 'w') as f:
#     pickle.dump(index2token,f)
#relation2index = dict((v, k) for k, v in index2relation.iteritems())

In [11]:
with open('../work/index2relation.pkl', 'r') as f:
    index2relation=pickle.load(f)
relation2index = dict((v, k) for k, v in index2relation.iteritems())

In [12]:
with open('../work/index2attribute.pkl', 'r') as f:
    index2attribute=pickle.load(f)
attribute2index = dict((v, k) for k, v in index2attribute.iteritems())

In [ ]:
image_id=attribute_train_image_ids[0]
image_objects=get_image_attributes(image_id,all_attributes)
img_file="../data/images/%d.jpg"%image_id
img=cv2.imread(img_file)
for image_object in image_objects:
    object_x=image_object[u'x']
    object_y=image_object[u'y']
    object_w=image_object[u'w']
    object_h=image_object[u'h']
    triple_attritbues=image_object[u'attributes']
    for attribute in triple_attritbues:
        train_attributes.append(attribute)

In [ ]:
img_file="../data/images/%d.jpg"%image_id

img=cv2.imread(img_file)



img= cv2.resize(img,(224,224))
img=img.transpose(2, 0, 1)-MEAN_VALUES
img=img.astype(xp.float32)


x_batch = np.ndarray((1, 3, 224,224), dtype=np.float32)
x_batch[0]=img
if gpu_id >=0:
    x = Variable(cuda.to_gpu(x_batch), volatile=True)
else:
    x = Variable(x_batch, volatile=True)
image_feature_chainer, = func(inputs={'data': x}, outputs=['pool5/7x7_s1'],
              disable=['loss1/ave_pool', 'loss2/ave_pool','loss3/classifier'],
              train=False)

image_feature_np=image_feature_chainer.data.reshape(1024)


In [16]:
def pre_extract_relation_feature(image_ids):
    triple_id2subject_feature={}
    triple_id2object_feature={}
    triple_id2relation_id={}
    for image_id in image_ids:
        image_relationships=get_image_relationships(image_id,all_relationships)
        img_file="../data/images/%d.jpg"%image_id
        img=cv2.imread(img_file,cv2.IMREAD_COLOR)
        all_img_list=[]
        triple_ids_tmp=[]
        for triple in image_relationships:
            triple_id=triple[u'id']
            triple_predicate=triple[u'predicate']

            if triple_predicate not in relation2index:
                continue
            relation_id=relation2index[triple_predicate]

            triple_subject=triple[u'subject']
            triple_subject_name=triple_subject['name']
            triple_subject_x=triple_subject['x']
            triple_subject_y=triple_subject['y']
            triple_subject_w=triple_subject['w']
            triple_subject_h=triple_subject['h']

            triple_object=triple[u'object']
            triple_object_name=triple_object['name']
            triple_object_x=triple_object['x']
            triple_object_y=triple_object['y']
            triple_object_w=triple_object['w']
            triple_object_h=triple_object['h']
            
            try:
                img_sbj=img[triple_subject_y:triple_subject_y+triple_subject_h, triple_subject_x:triple_subject_x+triple_subject_w]
                img_sbj=cv2.resize(img_sbj,(224,224))
                img_sbj=img_sbj.transpose(2, 0, 1)-MEAN_VALUES
                img_obj=img[triple_object_y:triple_object_y+triple_object_h, triple_object_x:triple_object_x+triple_object_w]
                img_obj=cv2.resize(img_obj,(224,224))
                img_obj=img_obj.transpose(2, 0, 1)-MEAN_VALUES
            except Exception as e:
                print 'image reading error'
                print 'type:' + str(type(e))
                print 'args:' + str(e.args)
                print 'message:' + e.message.strip()
                print 'image_id:'+ str(image_id)
                print 'triple_id:'+ str(triple_id)
                print ""
                continue
            triple_id2relation_id[triple_id]=relation_id
            all_img_list.append(img_sbj)
            all_img_list.append(img_obj)
            triple_ids_tmp.append(triple_id)

        x_batch = np.array(all_img_list, dtype=np.float32)
        if gpu_id >=0:
            x = Variable(cuda.to_gpu(x_batch), volatile=True)
        else:
            x = Variable(x_batch, volatile=True)
        image_feature_chainer, = func(inputs={'data': x}, outputs=['pool5/7x7_s1'],
                      disable=['loss1/ave_pool', 'loss2/ave_pool','loss3/classifier'],
                      train=False)

        shape=image_feature_chainer.data.shape[0:2]
        image_feature_xp=image_feature_chainer.data.reshape(shape)
        image_feature_np=cuda.to_cpu(image_feature_xp)

        for i in xrange(image_feature_np.shape[0]/2):
            triple_id=triple_ids_tmp[i]
            triple_id2subject_feature[triple_id]=image_feature_np[2*i]
            triple_id2object_feature[triple_id]=image_feature_np[2*i+1]

    return triple_id2subject_feature, triple_id2object_feature,triple_id2relation_id

In [17]:
image_id=relationship_train_image_ids[279]
pre_extract_relation_feature([image_id])
#triple_id2subject_feature, triple_id2object_feature,triple_id2relation_id=pre_extract_relation_feature([image_id])

InvalidType: 
Invalid operation is performed in: Convolution2DFunction (Forward)

Expect: in_types[0].ndim == 4
Actual: 1 != 4

In [18]:
image_id=relationship_train_image_ids[279]
for image_id in [image_id]:
    image_relationships=get_image_relationships(image_id,all_relationships)
    img_file="../data/images/%d.jpg"%image_id
    img=cv2.imread(img_file,cv2.IMREAD_COLOR)
    all_img_list=[]
    triple_ids_tmp=[]
    for triple in image_relationships:
        triple_id=triple[u'id']
        triple_predicate=triple[u'predicate']

        if triple_predicate not in relation2index:
            continue
        relation_id=relation2index[triple_predicate]

        triple_subject=triple[u'subject']
        triple_subject_name=triple_subject['name']
        triple_subject_x=triple_subject['x']
        triple_subject_y=triple_subject['y']
        triple_subject_w=triple_subject['w']
        triple_subject_h=triple_subject['h']

        triple_object=triple[u'object']
        triple_object_name=triple_object['name']
        triple_object_x=triple_object['x']
        triple_object_y=triple_object['y']
        triple_object_w=triple_object['w']
        triple_object_h=triple_object['h']

        try:
            img_sbj=img[triple_subject_y:triple_subject_y+triple_subject_h, triple_subject_x:triple_subject_x+triple_subject_w]
            img_sbj=cv2.resize(img_sbj,(224,224))
            img_sbj=img_sbj.transpose(2, 0, 1)-MEAN_VALUES
            img_obj=img[triple_object_y:triple_object_y+triple_object_h, triple_object_x:triple_object_x+triple_object_w]
            img_obj=cv2.resize(img_obj,(224,224))
            img_obj=img_obj.transpose(2, 0, 1)-MEAN_VALUES
        except Exception as e:
            print 'image reading error'
            print 'type:' + str(type(e))
            print 'args:' + str(e.args)
            print 'message:' + e.message.strip()
            print 'image_id:'+ str(image_id)
            print 'triple_id:'+ str(triple_id)
            print ""
            continue
        triple_id2relation_id[triple_id]=relation_id
        all_img_list.append(img_sbj)
        all_img_list.append(img_obj)
        triple_ids_tmp.append(triple_id)

    x_batch = np.array(all_img_list, dtype=np.float32)
    if gpu_id >=0:
        x = Variable(cuda.to_gpu(x_batch), volatile=True)
    else:
        x = Variable(x_batch, volatile=True)
    image_feature_chainer, = func(inputs={'data': x}, outputs=['pool5/7x7_s1'],
                  disable=['loss1/ave_pool', 'loss2/ave_pool','loss3/classifier'],
                  train=False)

    shape=image_feature_chainer.data.shape[0:2]
    image_feature_xp=image_feature_chainer.data.reshape(shape)
    image_feature_np=cuda.to_cpu(image_feature_xp)

    for i in xrange(image_feature_np.shape[0]/2):
        triple_id=triple_ids_tmp[i]
        triple_id2subject_feature[triple_id]=image_feature_np[2*i]
        triple_id2object_feature[triple_id]=image_feature_np[2*i+1]

InvalidType: 
Invalid operation is performed in: Convolution2DFunction (Forward)

Expect: in_types[0].ndim == 4
Actual: 1 != 4

In [145]:
image_ids=[image_id]

            

error: /home/travis/miniconda/conda-bld/work/opencv-3.1.0/modules/imgproc/src/imgwarp.cpp:3229: error: (-215) ssize.area() > 0 in function resize


In [19]:
x_batch

array([], dtype=float32)

In [149]:
img_obj.shape
print triple_object_x
print triple_object_y
print triple_object_w
print triple_object_h

-1
221
500
150


In [151]:
img_obj=img[triple_object_y:triple_object_y+triple_object_h, triple_object_x:triple_object_x+triple_object_w]

In [152]:
img_obj.shape

(150, 0, 3)

In [153]:
print triple_object_x
print triple_object_x+triple_object_w

-1
499


In [ ]:
def pre_extract_attribute_feature(image_ids):
    triple_id2subject_feature={}
    triple_id2attribute_id={}
    for image_id in image_ids:
        image_objects=get_image_attributes(image_id,all_attributes)
        img_file="../data/images/%d.jpg"%image_id
        img=cv2.imread(img_file)
        all_img_list=[]
        triple_ids_tmp=[]
        for image_object in image_objects:
            object_x=image_object[u'x']
            object_y=image_object[u'y']
            object_w=image_object[u'w']
            object_h=image_object[u'h']
            triple_attritbues=image_object[u'attributes']
            for attribute in triple_attritbues:
                train_attributes.append(attribute)
        
        image_relationships=get_image_relationships(image_id,all_relationships)

        for triple in image_relationships:
            triple_id=triple[u'id']
            triple_predicate=triple[u'predicate']

            if triple_predicate not in relation2index:
                continue

            relation_id=relation2index[triple_predicate]
            triple_id2relation_id[triple_id]=relation_id

            triple_ids_tmp.append(triple_id)

            triple_subject=triple[u'subject']
            triple_subject_name=triple_subject['name']
            triple_subject_x=triple_subject['x']
            triple_subject_y=triple_subject['y']
            triple_subject_w=triple_subject['w']
            triple_subject_h=triple_subject['h']
            img_sbj=img[triple_subject_y:triple_subject_y+triple_subject_h, triple_subject_x:triple_subject_x+triple_subject_w]
            img_sbj=cv2.resize(img_sbj,(224,224))
            img_sbj=img_sbj.transpose(2, 0, 1)-MEAN_VALUES
            all_img_list.append(img_sbj)

            triple_object=triple[u'object']
            triple_object_name=triple_object['name']
            triple_object_x=triple_object['x']
            triple_object_y=triple_object['y']
            triple_object_w=triple_object['w']
            triple_object_h=triple_object['h']
            img_obj=img[triple_object_y:triple_object_y+triple_object_h, triple_object_x:triple_object_x+triple_object_w]
            img_obj=cv2.resize(img_obj,(224,224))
            img_obj=img_obj.transpose(2, 0, 1)-MEAN_VALUES
            all_img_list.append(img_obj)

        x_batch = np.array(all_img_list, dtype=np.float32)
        if gpu_id >=0:
            x = Variable(cuda.to_gpu(x_batch), volatile=True)
        else:
            x = Variable(x_batch, volatile=True)
        image_feature_chainer, = func(inputs={'data': x}, outputs=['pool5/7x7_s1'],
                      disable=['loss1/ave_pool', 'loss2/ave_pool','loss3/classifier'],
                      train=False)

        shape=image_feature_chainer.data.shape[0:2]
        image_feature_xp=image_feature_chainer.data.reshape(shape)
        image_feature_np=cuda.to_cpu(image_feature_xp)

        for i in xrange(image_feature_np.shape[0]/2):
            triple_id=triple_ids_tmp[i]
            triple_id2subject_feature[triple_id]=image_feature_np[2*i]
            triple_id2object_feature[triple_id]=image_feature_np[2*i+1]

    return triple_id2subject_feature,triple_id2attribute_id

In [ ]:
image_id=attribute_train_image_ids[0]
triple_id2subject_feature,triple_id2attribute_id=pre_extract_attribute_feature([image_id])